<a href="https://colab.research.google.com/github/Birkbeck/msc-projects-2023-4-Gabriele_Monti_PEFT/blob/main/Synthetic_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Synthetic evaluation**
In this experiment, we try to evaluate a Gemma fine-tuned with Commodore 64 question-answer pairs to train a potential technical bot. First, the model is used to generate the answers. Then a synthetic evaluation is performed. The answers are evaluated by another model.

A second experiment uses the questions, but the Gemma fine-tuned model is also given the context. Then it is synthetically evaluated. A third experiment uses the original Gemma model without any fine-tuning, and then it is evaluated again.

The findings reveal that the fine-tuned model performs very badly at the task. It improves when it is given the context of the question, but there is no difference between the fine-tuned and the original Gemma. The conclusion is that using this type of fine-tuning (wide knowledge chatbot) requires more investigation. Expanding the dataset and method requires are field open for further experimentation.




In [ ]:
#!pip install peft
#!pip install transformers
!pip install --upgrade peft

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
! pip install datasets

In [ ]:
from huggingface_hub import notebook_login

notebook_login()
#xxxxxxx

In [ ]:
#load the commodore 64 daatset

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset
import random

# Load the dataset
dataset = load_dataset('theoracle/commodore64')



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/1832 [00:00<?, ? examples/s]

In [ ]:
question

'What is the reference point for calculating the position of a sprite in a 24 by 21 dot area and what is the cost of the register?'

In [ ]:
#load the commodore fine tuned model

from transformers import AutoModelForCausalLM, AutoTokenizer

model_path = "theoracle/autotrain-lsmbq-gemma64"

offload_dir = "/content/Untitled Folder"

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    device_map="auto",
    torch_dtype='auto',
   # offload_dir=offload_dir
).eval()

tokenizer.pad_token = tokenizer.eos_token
# Your prompt content

tokenizer_config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/555 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/689 [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/78.5M [00:00<?, ?B/s]

In [ ]:
#take 100 random questions from the dataset and generate the model response

import pandas as pd
# Initialize a list to store results
results = []

# Loop through 100 questions
for i in range(100):
    # Select a random question
    random_entry = random.choice(dataset['train'])  # Assuming you're using the 'train' split
    question = random_entry['question']
    chunks = random_entry['chunks']
    summary = random_entry['summary']

    # Define the prompt
    prompt = f"###human: {question} ###\n bot:"

    # Tokenize the prompt to get input IDs and attention mask
    encoding = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True, max_length=500, add_special_tokens=True)
    input_ids = encoding['input_ids']
    attention_mask = encoding['attention_mask']

    # Generate output, ensuring to pass the attention mask and set pad_token_id
    output_ids = model.generate(
        input_ids.to('cuda'),
        attention_mask=attention_mask.to('cuda'),
        max_new_tokens=100,  # Specify the number of new tokens to generate
        pad_token_id=tokenizer.eos_token_id,
        #temperature=0.7,
        #do_sample=True,
        #top_k=1,
        #repetition_penalty=1.5
    )

    # Decode the generated response
    response = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    # Save the question and response in a dictionary
    results.append({
        'question': question,
        'response': response,
        'chunks': chunks,
        'summary': summary
    })

# Convert the results list to a DataFrame
df = pd.DataFrame(results)

# Save the DataFrame to a CSV file
df.to_csv('model_responses.csv', index=False)

print(df.head())  # Print the first few entries of the DataFrame

                                            question  \
0  What are the instructions for in the input/out...   
1  What does the command 'pokevic+0,100' do accor...   
2  What does the second byte of the instruction p...   
3         What does F5 represent in this given text?   
4  What does the term 'BASIC TO MACHINE LANGUAGE'...   

                                            response  \
0  ###human: What are the instructions for in the...   
1  ###human: What does the command 'pokevic+0,100...   
2  ###human: What does the second byte of the ins...   
3  ###human: What does F5 represent in this given...   
4  ###human: What does the term 'BASIC TO MACHINE...   

                                              chunks  \
0  g/creating\n  only the buffer(s) required. Whe...   
1  176 printtab(55)"on top of each other"\n  180 ...   
2  t memory loca-\n  tion in page zero contains t...   
3         D3 - Future Expansion           F3 - Fu...   
4  H    00BF          191        Serial Word B

In [ ]:
df

,question,response,chunks,summary
0,What sequence comes after the letters A B C D ...,###human: What sequence comes after the letter...,| |\n | 12 | GND | ...,"GND, N, GND, A B C D E F H J K L M N, 1 1, 2, ..."
1,What does the example program in tok64 page187...,###human: What does the example program in tok...,ree voices singing.\n This example program ...,The example program shows you one way to trans...
2,What is the function of LORAM (bit 0) in the o...,###human: What is the function of LORAM (bit 0...,ves a value of 47 decimal. The Commodore 64 au...,The Commodore 64 automatically sets the data d...
3,What standard code do most computers use for I...,###human: What standard code do most computers...,ify the parity and duplex of the transmission....,See RS-232 section or VICMODEM manual for deta...
4,What happens to a sprite when you create it an...,###human: What happens to a sprite when you cr...,\n CHART for more information.)\n When you...,"When you create a sprite, it stays in memory u..."
5,What is missing from the text on page 23 of th...,###human: What is missing from the text on pag...,23 /CS\n | ...,The picture of Block Diggram is missing from t...
6,What is the significance of the numerals on C#...,###human: What is the significance of the nume...,----------------------+\n | MUSICAL NO...,"On the musical note, OSCILLATOR FREQ is the so..."
7,What happens when bit 3 of the DDR is set to 1...,###human: What happens when bit 3 of the DDR i...,", the corresponding line\n of the PORT will b...","To PEEK or POKE the USER port, it is necessary..."
8,What can clear the latch in the dynamic ram re...,###human: What can clear the latch in the dyna...,"been set, the latch may be cleared only\n by...",The dynamic ram refresh controller is built in...
9,What changes occur to the Y parameters and spr...,###human: What changes occur to the Y paramete...,ange to she following:\n\n\n ...,"In the 24 row mode, the Y parameters change to..."


In [ ]:
!pip install openai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 361.5/361.5 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.2 MB/s eta 0:00:00


In [ ]:
#use gpt-4o-mini to check the model answers

from openai import OpenAI
client = OpenAI(api_key="xxxxxxxxxxxxxxxxxxxxxxxx")  # Replace with your actual API key

# Function to query OpenAI
def query_openai(prompt):
    response = client.chat.completions.create(
        model="gpt-4o-mini",  # Specify the model you are using
        messages=[{"role": "user", "content": prompt}],
        max_tokens=50,
        temperature=0
    )
    return response

# Iterate through each row in the DataFrame and check if the response contains the correct answer
results = []
for index, row in df.iterrows():
    prompt = f"You are a classifier; given this context '{row['summary']}', check if the generated answer '{row['response']}' contains a correct answer to the question. Output a value 1 for True or 0 for False.:"
    output = query_openai(prompt)

    # Extract the result from the API output
    result = output.choices[0].message.content.strip()  # Use dot notation to access attributes

    results.append(result)

# Optionally, add the results back to the DataFrame
df['correct_answer_check'] = results

# Save the DataFrame with the correctness check to a CSV file
df.to_csv('model_responses_with_checks.csv', index=False)

# Print the results for each row
for result in results:
    print(result)


0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
1
0
1
0
0
0
0
0
1
0
0
1
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
1
0
0
0
0
0
0


In [ ]:
df

,question,response,chunks,summary,correct_answer_check
0,What are the instructions for in the input/out...,###human: What are the instructions for in the...,g/creating\n only the buffer(s) required. Whe...,BASIC programs are expected to run at the same...,0
1,"What does the command 'pokevic+0,100' do accor...","###human: What does the command 'pokevic+0,100...","176 printtab(55)""on top of each other""\n 180 ...","Pokevic set sprite 0's x position, sprite 1's ...",0
2,What does the second byte of the instruction p...,###human: What does the second byte of the ins...,t memory loca-\n tion in page zero contains t...,"In indirect indexed addressing, the second byt...",0
3,What does F5 represent in this given text?,###human: What does F5 represent in this given...,D3 - Future Expansion F3 - Fu...,"D3 - Future Expansion, F3 - SBC - Zero Page,X,...",0
4,What does the term 'BASIC TO MACHINE LANGUAGE'...,###human: What does the term 'BASIC TO MACHINE...,H 00BF 191 Serial Word Buff...,The current key is H. The current key pressed ...,1
...,...,...,...,...,...
95,What does the PEEK function return and what is...,###human: What does the PEEK function return a...,PEEK\n\n\n TYPE: Integer Function\n FORMAT:...,The PEEK function returns an integer in the ra...,0
96,What does line 20 in the program set for the C...,###human: What does line 20 in the program set...,s come from the SPRITEMAKING CHART on Page 176...,The program defines the first sprite as a soli...,0
97,What defines the location of the video matrix ...,###human: What defines the location of the vid...,tion addresses in the 2048 byte CHARACTER BASE...,The video matrix is comprised of 1000 consecut...,0
98,What does register 6 control in this system?,###human: What does register 6 control in this...,+------|-------\n ...,register 24 determines which type filter you ...,0


In [ ]:
#check how many correct answer we have
df['correct_answer_check'] = df['correct_answer_check'].astype(int)

# Count the number of '1's in the 'correct_answer_check' column
count_true = df['correct_answer_check'].sum()

print("Number of '1's in the 'correct_answer_check' column:", count_true)

Number of '1's in the 'correct_answer_check' column: 9


In [ ]:
# use the fine tuned model again but aslo add the context to the question

import pandas as pd
# Initialize a list to store results
results = []

# Loop through 100 questions
for i in range(100):
    # Select a random question
    random_entry = random.choice(dataset['train'])  # Assuming you're using the 'train' split
    question = random_entry['question']
    chunks = random_entry['chunks']
    summary = random_entry['summary']

    # Define the prompt
    prompt = f"###human: given this context {chunks} answer this {question} ###\n bot:"

    # Tokenize the prompt to get input IDs and attention mask
    encoding = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True, max_length=500, add_special_tokens=True)
    input_ids = encoding['input_ids']
    attention_mask = encoding['attention_mask']

    # Generate output, ensuring to pass the attention mask and set pad_token_id
    output_ids = model.generate(
        input_ids.to('cuda'),
        attention_mask=attention_mask.to('cuda'),
        max_new_tokens=100,  # Specify the number of new tokens to generate
        pad_token_id=tokenizer.eos_token_id,
        #temperature=0.7,
        #do_sample=True,
        #top_k=1,
        #repetition_penalty=1.5
    )

    # Decode the generated response
    response = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    # Save the question and response in a dictionary
    results.append({
        'question': question,
        'response': response,
        'chunks': chunks,
        'summary': summary
    })

# Convert the results list to a DataFrame
df2 = pd.DataFrame(results)

# Save the DataFrame to a CSV file
df2.to_csv('model_responses.csv', index=False)

print(df2.head())  # Print the first few entries of the DataFrame

                                            question  \
0  What does the ULTIMAX video game memory map in...   
1  What does Commodore SEMICONDUCTOR GROUP allow ...   
2  What is the total amount of power dissipation ...   
3  How many characters can the Commodore 64's key...   
4  What will happen if you hold down the <CTRL> k...   

                                            response  \
0  ###human: given this context \n        (9)   T...   
1  ###human: given this context e I/O Pins as inp...   
2  ###human: given this context ---+-------+-----...   
3  ###human: given this context g\n  features to ...   
4  ###human: given this context he upper left\n  ...   

                                              chunks  \
0  \n        (9)   This is the ULTIMAX video game...   
1  e I/O Pins as inputs (using the Data Direction...   
2  ---+-------+-----+\n  | Power Dissipation     ...   
3  g\n  features to your programs.\n\n\n\n\n\n\n ...   
4  he upper left\n  corner of the screen, you 

In [ ]:
df2

,question,response,chunks,summary,correct_answer_check
0,What does the ULTIMAX video game memory map in...,###human: given this context \n (9) T...,\n (9) This is the ULTIMAX video game...,The ULTIMAX video game memory map shows that t...,1
1,What does Commodore SEMICONDUCTOR GROUP allow ...,###human: given this context e I/O Pins as inp...,e I/O Pins as inputs (using the Data Direction...,Commodore SEMICONDUCTOR GROUP reserves the rig...,The generated answer incorrectly states that t...
2,What is the total amount of power dissipation ...,###human: given this context ---+-------+-----...,---+-------+-----+\n | Power Dissipation ...,Power Dissipation is the total amount of elect...,The generated answer does contain a correct an...
3,How many characters can the Commodore 64's key...,###human: given this context g\n features to ...,g\n features to your programs.\n\n\n\n\n\n\n ...,The Commodore 64 has a keyboard buffer that ho...,1
4,What will happen if you hold down the <CTRL> k...,###human: given this context he upper left\n ...,"he upper left\n corner of the screen, you wou...",Holding down the <CTRL> key and hitting <9> wi...,0
...,...,...,...,...,...
95,What does Step 3 instruct to do in pixel sprit...,###human: given this context [THE ...,[THE PICTURE IS MISSING!]\n\n\n\n\...,Step 1: Color in the pixels on the spritemakin...,1
96,What are the corresponding rules that the read...,###human: given this context 120 readnm\n\n\...,120 readnm\n\n\n\n\n\n\n\n\n 130 ifnm=0then...,"There are 120 readnm, 500 pokes, 5,0:pokes, 6,...",1
97,What happens when the signal is set to zero or...,###human: given this context 0-3 determine whi...,0-3 determine which signals will be\n routed ...,0-3 determine which signals will be routed thr...,1
98,What is the exact position of a sprite determi...,###human: given this context NOT\n matter how...,NOT\n matter how many or how few dots you use...,There are 200 differences between vertical and...,1


In [ ]:
#use again gpt-4o-mini to check the answers

from openai import OpenAI
client = OpenAI(api_key="xxxxx")  # Replace with your actual API key

# Function to query OpenAI
def query_openai(prompt):
    response = client.chat.completions.create(
        model="gpt-4o-mini",  # Specify the model you are using
        messages=[{"role": "user", "content": prompt}],
        max_tokens=50,
        temperature=0
    )
    return response

# Iterate through each row in the DataFrame and check if the response contains the correct answer
results = []
for index, row in df2.iterrows():
    prompt = f"You are a classifier; given this context '{row['summary']}', check if the generated answer '{row['response']}' contains a correct answer to the question. Output a value 1 for True or 0 for False.:"
    output = query_openai(prompt)

    # Extract the result from the API output
    result = output.choices[0].message.content.strip()  # Use dot notation to access attributes

    results.append(result)

# Optionally, add the results back to the DataFrame
df2['correct_answer_check'] = results

# Save the DataFrame with the correctness check to a CSV file
df2.to_csv('model_responses_with_checks.csv', index=False)

# Print the results for each row
for result in results:
    print(result)


1
The generated answer incorrectly states that the user can change the contents of the "Output Register" at address 0001, while the original context specifies that the user can change the contents of the "Data Direction Register" at that address. Therefore, the
The generated answer does contain a correct answer to the question about the total amount of power dissipation according to the text, which is 6581 mW. Therefore, the output value is 1 for True.
1
0
The generated answer does not contain a correct answer to the question based on the provided context. The context describes a program with specific lines of code and their functions, but the generated answer includes modifications and additional lines that are not present in the original context.
1
0
1
1
0
0
0
0
0
1
1
1
1
0
1
0
0
0
0
0
0
0
1
1
To determine if the generated answer contains a correct answer to the question "What key routine must be called to update the time in this system?", we need to analyze the context provided and 

In [ ]:
# clean up the answers that do not contain 1 or 0 onlu
import pandas as pd

# Assuming 'df2' is your DataFrame
# Remove rows where 'correct_answer_check' is not '1' or '0'
df2_filtered = df2[df2['correct_answer_check'].isin(['1', '0'])]

# Convert the 'correct_answer_check' column to integers
df2_filtered['correct_answer_check'] = df2_filtered['correct_answer_check'].astype(int)

# Count the number of '1's in the 'correct_answer_check' column
count_true = df2_filtered['correct_answer_check'].sum()

print("Number of '1's in the 'correct_answer_check' column:", count_true)


Number of '1's in the 'correct_answer_check' column: 58


<ipython-input-58-5c6d9850082a>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_filtered['correct_answer_check'] = df2_filtered['correct_answer_check'].astype(int)


In [ ]:
df2_filtered

,question,response,chunks,summary,correct_answer_check
0,What does the ULTIMAX video game memory map in...,###human: given this context \n (9) T...,\n (9) This is the ULTIMAX video game...,The ULTIMAX video game memory map shows that t...,1
3,How many characters can the Commodore 64's key...,###human: given this context g\n features to ...,g\n features to your programs.\n\n\n\n\n\n\n ...,The Commodore 64 has a keyboard buffer that ho...,1
4,What will happen if you hold down the <CTRL> k...,###human: given this context he upper left\n ...,"he upper left\n corner of the screen, you wou...",Holding down the <CTRL> key and hitting <9> wi...,0
6,What is the equation used to determine the fre...,"###human: given this context sters 00,01)\n\n\...","sters 00,01)\n\n\n Together these registers...",Oscillator 1's frequency is determined by the ...,1
7,"What does the OP Code, '8E', in the Assembly L...",###human: given this context --+--------------...,--+-----------------------+---------+---------...,"The Addressing Mode, Assembly Language Form, O...",0
...,...,...,...,...,...
95,What does Step 3 instruct to do in pixel sprit...,###human: given this context [THE ...,[THE PICTURE IS MISSING!]\n\n\n\n\...,Step 1: Color in the pixels on the spritemakin...,1
96,What are the corresponding rules that the read...,###human: given this context 120 readnm\n\n\...,120 readnm\n\n\n\n\n\n\n\n\n 130 ifnm=0then...,"There are 120 readnm, 500 pokes, 5,0:pokes, 6,...",1
97,What happens when the signal is set to zero or...,###human: given this context 0-3 determine whi...,0-3 determine which signals will be\n routed ...,0-3 determine which signals will be routed thr...,1
98,What is the exact position of a sprite determi...,###human: given this context NOT\n matter how...,NOT\n matter how many or how few dots you use...,There are 200 differences between vertical and...,1


In [ ]:
#load the gemma model without any fine tuning and geneate the the answers with context

from transformers import AutoModelForCausalLM, AutoTokenizer


model_path = "google/gemma-2b-it"

offload_dir = "/content/Untitled Folder"

tokenizer = AutoTokenizer.from_pretrained(model_path)
model2 = AutoModelForCausalLM.from_pretrained(
    model_path,
    device_map="auto",
    torch_dtype='auto',
   # offload_dir=offload_dir
).eval()

tokenizer.pad_token = tokenizer.eos_token
# Your prompt content

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
import pandas as pd
# Initialize a list to store results
results = []

# Loop through 100 questions
for i in range(100):
    # Select a random question
    random_entry = random.choice(dataset['train'])  # Assuming you're using the 'train' split
    question = random_entry['question']
    chunks = random_entry['chunks']
    summary = random_entry['summary']

    # Define the prompt
    prompt = f"###human: given this context {chunks} answer this {question} ###\n bot:"

    # Tokenize the prompt to get input IDs and attention mask
    encoding = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True, max_length=500, add_special_tokens=True)
    input_ids = encoding['input_ids']
    attention_mask = encoding['attention_mask']

    # Generate output, ensuring to pass the attention mask and set pad_token_id
    output_ids = model2.generate(
        input_ids.to('cuda'),
        attention_mask=attention_mask.to('cuda'),
        max_new_tokens=100,  # Specify the number of new tokens to generate
        pad_token_id=tokenizer.eos_token_id,
        #temperature=0.7,
        #do_sample=True,
        #top_k=1,
        #repetition_penalty=1.5
    )

    # Decode the generated response
    response = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    # Save the question and response in a dictionary
    results.append({
        'question': question,
        'response': response,
        'chunks': chunks,
        'summary': summary
    })

# Convert the results list to a DataFrame
df3 = pd.DataFrame(results)

# Save the DataFrame to a CSV file
df3.to_csv('model_responses.csv', index=False)

print(df3.head())  # Print the first few entries of the DataFrame

                                            question  \
0  What happens to the value of TI$ after tape I/...   
1  What is the function of Register 25 in relatio...   
2  What alphabetic letters do the 'o' and 'x' in ...   
3  What are some examples of array names, value a...   
4  What does 0 and 1 represent in the context of ...   

                                            response  \
0  ###human: given this context nds. The TI$ time...   
1  ###human: given this context  the oscillator a...   
2  ###human: given this context   1  2  3  4  5  ...   
3  ###human: given this context y. Values outside...   
4  ###human: given this context RQTMP  029F-02A0 ...   

                                              chunks  \
0  nds. The TI$ timer can\n  also be assigned an ...   
1   the oscillator and is not affected in any way...   
2    1  2  3  4  5  6  7  8  9 10 11 12 13 14 15\...   
3  y. Values outside that range will cause\n  the...   
4  RQTMP  029F-02A0     671-672    Holds IRQ V

In [ ]:
#check the answers of the gemma model with gpt-4o-mini

from openai import OpenAI
client = OpenAI(api_key="XXXX")  # Replace with your actual API key

# Function to query OpenAI
def query_openai(prompt):
    response = client.chat.completions.create(
        model="gpt-4o-mini",  # Specify the model you are using
        messages=[{"role": "user", "content": prompt}],
        max_tokens=50,
        temperature=0
    )
    return response

# Iterate through each row in the DataFrame and check if the response contains the correct answer
results = []
for index, row in df3.iterrows():
    prompt = f"You are a classifier; given this context '{row['summary']}', check if the generated answer '{row['response']}' contains a correct answer to the question. Output a value 1 for True or 0 for False.:"
    output = query_openai(prompt)

    # Extract the result from the API output
    result = output.choices[0].message.content.strip()  # Use dot notation to access attributes

    results.append(result)

# Optionally, add the results back to the DataFrame
df3['correct_answer_check'] = results

# Save the DataFrame with the correctness check to a CSV file
df3.to_csv('model_responses_with_checks.csv', index=False)

# Print the results for each row
for result in results:
    print(result)


1
1
0
1
0
1
0
0
0
1
1
1
0
1
0
1
0
1
1
1
0
1
1
1
0
1
1
1
0
0
0
0
0
1
1
1
1
0
1
0
1
0
0
1
0
1
0
1
1
1
0
0
1
0
0
1
1
0
0
1
The generated answer does contain a correct answer to the question regarding the function of the JSR CLALL routine in relation to open files and the open file table. Therefore, the output value is:

1
1
1
0
1
1
1
1
1
1
1
0
1
1
1
0
0
0
0
1
0
0
1
1
1
0
0
0
0
1
0
1
1
1
0
1
1
0
0
0


In [ ]:
import pandas as pd

# Assuming 'df2' is your DataFrame
# Remove rows where 'correct_answer_check' is not '1' or '0'
df3_filtered = df3[df3['correct_answer_check'].isin(['1', '0'])]

# Convert the 'correct_answer_check' column to integers
df3_filtered['correct_answer_check'] = df3_filtered['correct_answer_check'].astype(int)

# Count the number of '1's in the 'correct_answer_check' column
count_true = df3_filtered['correct_answer_check'].sum()

print("Number of '1's in the 'correct_answer_check' column:", count_true)

Number of '1's in the 'correct_answer_check' column: 55


<ipython-input-70-67bf612600b0>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3_filtered['correct_answer_check'] = df3_filtered['correct_answer_check'].astype(int)


In [ ]:
df3_filtered

,question,response,chunks,summary,correct_answer_check
0,What happens to the value of TI$ after tape I/...,###human: given this context nds. The TI$ time...,nds. The TI$ timer can\n also be assigned an ...,The TI$ timer can be assigned an arbitrary sta...,1
1,What is the function of Register 25 in relatio...,###human: given this context the oscillator a...,the oscillator and is not affected in any way...,Register 25 gives you access to the output of...,1
2,What alphabetic letters do the 'o' and 'x' in ...,###human: given this context 1 2 3 4 5 ...,1 2 3 4 5 6 7 8 9 10 11 12 13 14 15\...,"1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14 ...",0
3,"What are some examples of array names, value a...",###human: given this context y. Values outside...,y. Values outside that range will cause\n the...,Y. y. values outside the range will cause the ...,1
4,What does 0 and 1 represent in the context of ...,###human: given this context RQTMP 029F-02A0 ...,RQTMP 029F-02A0 671-672 Holds IRQ Vect...,"RQTMP, ENABL, RS-232 Enables, D1 IRQ Indicator...",0
...,...,...,...,...,...
95,What term is commonly used to describe the num...,###human: given this context s. Think of throw...,s. Think of throwing a\n stone into a pool an...,"When similar waves are created in air, we hear...",1
96,Can the listening routine be used to send seco...,###human: given this context address: $FF93 (h...,address: $FF93 (hex) 65427 (decimal)\n Comm...,Listening routine sends a secondary address to...,1
97,What are the normal BASIC commands used by the...,###human: given this context p in the standard...,"p in the standard RS-232 format, but\n the vo...",The RS-232 on the BASIC level uses the normal ...,0
98,What allows the Z-80 cartridge to translate th...,###human: given this context ut you can NOT us...,ut you can NOT use them at the same time in a\...,The Z-80 cartridge translates the memory addre...,0
